# FCN VOC 2012 and SBD Semi Supervised Learning

In [1]:
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision
from torchvision import models
import torch.utils.data as tud
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.utils as vutils
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm import tqdm
from PIL import Image
from collections import Counter
from sklearn.metrics import jaccard_score
import pickle
import my_datasets as mdset
import eval_train as ev
from utils import * 




## Parameters

In [2]:
#MODEL SAVE AND LOAD 
dataroot_voc = '/data/voc2012'
dataroot_sbd = '/data/sbd'
model_name = 'fcn_voc_sbd30_semisup_g09_KL' # Model name for save during semisup training
SAVE_DIR = '/data/model'
file_base = None # Base model 
fcn= True
pretrained=False
save_all_ep = True # A save for each epoch 
save = os.path.join(SAVE_DIR,model_name)
# MODEL PARAMETERS 
desactivate_bn = False # Desactive BatchNorm for fine tune models. 
# TRAIN PARAMETERS
batch_size = 2
gamma = 0.9
learning_rate = 10e-4
moment = 0.9
wd = 2e-4
n_epochs = 25 # N_epoch for multi_task/semi sup training
n_epochs_supervised = 20 # Train in fully supervised for n_epoch_supervised
angle_max = 30

# LOSS 

#criterion_unsupervised = nn.L1Loss(reduction='mean')
criterion_unsupervised = nn.KLDivLoss(reduction = 'batchmean', log_target = False)
#criterion_unsupervised = nn.CrossEntropyLoss(ignore_index=21)
criterion_supervised = nn.CrossEntropyLoss(ignore_index=21) # On ignore la classe border.
Loss = 'KL' # Loss = 'KL' or 'CE' or None for L1,MSE…

# DATASET AND DATA AUG 
rotate = False # random rotation during training
split = True # split the supervised dataset
rate = 0.3 # percent of data of VOC + SBD we use for supervised dataset
fully_supervised = False # Use the same dataloader for equivariance loss and supervised loss

# BEFORE TRAIN IN SEMISUP/MULTITASK
pretrain_model = False  # Pretrain the model in a fully supervised way before appli equiv loss
eval_before_semisup = False # Eval the loaded model before training with equiv_loss 

## DATASETS

In [3]:
train_dataset_VOC = mdset.VOCSegmentation(dataroot_voc,year='2012', image_set='train', download=True,rotate=rotate)
val_dataset_VOC = mdset.VOCSegmentation(dataroot_voc,year='2012', image_set='val', download=True)
train_dataset_SBD = mdset.SBDataset(dataroot_sbd, image_set='train_noval',mode='segmentation',rotate=rotate)

Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar


### Concatene Dataset

In [4]:
train_dataset_unsup = tud.ConcatDataset([train_dataset_VOC,train_dataset_SBD])

### Split dataset

In [5]:
if split:
    train_dataset_sup = split_dataset(train_dataset_unsup,rate)

In [6]:
if fully_supervised : 
    train_dataset_unsup = train_dataset_sup

dataloader_train_VOC = torch.utils.data.DataLoader(train_dataset_VOC, batch_size=batch_size,\
                                                       shuffle=True,drop_last=True)
dataloader_train_sup = torch.utils.data.DataLoader(train_dataset_sup, batch_size=batch_size,\
                                                       shuffle=True,drop_last=True)
dataloader_train_equiv = torch.utils.data.DataLoader(train_dataset_unsup,batch_size=batch_size,\
                                                     shuffle=True,drop_last=True)

dataloader_val = torch.utils.data.DataLoader(val_dataset_VOC, batch_size=batch_size)
# Decide which device we want to run on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device :",device)



device : cuda:0


In [7]:
print("Taille dataset train supervised :",len(train_dataset_sup))
print("Taille dataset train unsupervised :",len(train_dataset_unsup))
print("Taille dataset val VOC :",len(val_dataset_VOC))

Taille dataset train supervised : 2126
Taille dataset train unsupervised : 7087
Taille dataset val VOC : 1449



## Load Pytorch Model

In [8]:
def load_model(file=None,fcn=False,pretrained=False):
    if file is None:
        if fcn is False:
            model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=pretrained)
        else:
            model = torchvision.models.segmentation.fcn_resnet101(pretrained=pretrained)
    else:
        model = torch.load(os.path.join(SAVE_DIR,file))
    return model

In [9]:
model = load_model(file=file_base,fcn=fcn,pretrained=pretrained)

In [10]:
model.to(device)

FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

## Training


In [11]:
iou_train = []
iou_test = []
combine_loss_train = []
combine_loss_test = []
loss_train_unsup = []
loss_train_sup = []
loss_test = []
loss_test_unsup = []
pix_accuracy_train = []
pix_accuracy_test = []
accuracy_test = []
accuracy_train = []
#
all_combine_loss_train = []
all_loss_train_sup = []
all_loss_train_unsup = []
all_iou_train= []
all_pix_accuracy =  []

In [12]:
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=moment, weight_decay=wd)

### Pretrain fully supervised 

In [13]:
## pretrain the model in fully supervised
if pretrain_model:
    torch.autograd.set_detect_anomaly(True)
    for ep in range(n_epochs_supervised):
        
        print("EPOCH",ep)
        model.train()
        if desactivate_bn : 
            model.apply(deactivate_batchnorm) # Disable BN 
        for i,(x,mask) in enumerate(dataloader_train_sup):
            x = x.to(device)
            mask = mask.to(device)  
            pred = model(x)
            pred = pred["out"]
            loss = criterion_supervised(pred,mask)
            all_loss_train_sup.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            all_iou_train.append(inter_over_union(pred.argmax(dim=1).detach().cpu(),mask.detach().cpu()))
            optimizer.step()        

        #lr_scheduler.step()
        m_iou = np.array(all_iou_train).mean()
        m_loss = np.array(all_loss_train_sup).mean()
        loss_train_sup.append(m_loss)
        iou_train.append(m_iou)
        all_loss_train_sup = []
        all_iou_train = []
        print("EP:",ep," loss train:",m_loss," iou train:",m_iou)

        #Eval model

        model.eval()
        state = ev.eval_model(model,dataloader_val,device=device,num_classes=21)
        iou = state.metrics['mean IoU']
        acc = state.metrics['accuracy']
        loss = state.metrics['CE Loss'] 
        loss_test.append(loss)
        iou_test.append(iou)
        accuracy_test.append(acc)
        print('EP:',ep,'iou:',state.metrics['mean IoU'],\
              'Accuracy:',state.metrics['accuracy'],'Loss CE',state.metrics['CE Loss'])

        torch.save(model,save)
        ## Save model
        if save_all_ep:
            save_model = model_name+'_pretrain_sup'+'_ep'+str(ep)+'.pt'
            save = os.path.join(SAVE_DIR,save_model)
            torch.save(model,save)
        else:
            save_model = model_name+'_pretrain_sup'+'.pt'
            save = os.path.join(SAVE_DIR,save_model)
            torch.save(model,save)
    

In [14]:
if eval_before_semisup:
    model.eval()
    print("EVAL ON TRAIN VOC 2012:")
    m_pix_acc, m_loss_equiv = eval_accuracy_equiv(model,dataloader_train_VOC,criterion=criterion_unsupervised,\
                            nclass=21,device=device,Loss=Loss,plot=False,angle_max=30,random_angle=False) 
    d_iou = ev.eval_model_all_angle(model,train=True,batch_size=batch_size,device=device,num_classes=21)
    print("EVAL ON VAL VOC 2012:")
    m_pix_acc, m_loss_equiv = eval_accuracy_equiv(model,dataloader_val,criterion=criterion_unsupervised,\
                            nclass=21,device=device,Loss=Loss,plot=False,angle_max=30,random_angle=False) 
    d_iou = ev.eval_model_all_angle(model,train=False,batch_size=batch_size,device=device,num_classes=21)


In [15]:
learning_rate = 2*learning_rate

### Semi sup training

In [16]:
torch.autograd.set_detect_anomaly(True)
#torch.backends.cudnn.benchmark=True
for ep in range(n_epochs):
    #dataloader_train_sup = torch.utils.data.DataLoader(train_dataset_sup, batch_size=batch_size,\
                                                      # shuffle=True,drop_last=True)
    dataloader_train_equiv = torch.utils.data.DataLoader(train_dataset_unsup,batch_size=batch_size,\
                                                     shuffle=True,drop_last=True)
    print("EPOCH",ep)
    model.train()
    if desactivate_bn : 
        model.apply(deactivate_batchnorm) # Disable BN 
    for batch_sup,batch_unsup in zip(dataloader_train_sup,dataloader_train_equiv):
        optimizer.zero_grad()
        if random.random() > 0.5: # I use this to rotate the image on the left and on the right during training.
            angle = np.random.randint(0,angle_max)
        else:
            angle = np.random.randint(360-angle_max,360)
        x_unsup,_ = batch_unsup
        loss_equiv,acc = compute_transformations_batch(x_unsup,model,angle,reshape=False,\
                                                     criterion=criterion_unsupervised,Loss = Loss,\
                                                       device=device)
        x,mask = batch_sup
        x = x.to(device)
        mask = mask.to(device)
        pred = model(x)["out"]
        loss_equiv = loss_equiv.to(device) # otherwise bug in combining the loss 
        loss_sup = criterion_supervised(pred,mask)
        loss = gamma*loss_sup + (1-gamma)*loss_equiv # combine loss              
        loss.backward()
        optimizer.step()
        
        # append for plot
        all_pix_accuracy.append(acc) # accuracy between the original mask and the transform mask put back in place
        all_loss_train_unsup.append(loss_equiv.item())
        all_loss_train_sup.append(loss_sup.item())
        all_combine_loss_train.append(loss.item())
            
    #lr_scheduler.step()
    #
    # Eval IoU Train 
    state = ev.eval_model(model,dataloader_val,device=device,num_classes=21)
    iou = state.metrics['mean IoU']
    iou_train.append(iou)
    m_loss_combine = np.array(all_combine_loss_train).mean()
    m_acc = np.array(all_pix_accuracy).mean()
    combine_loss_train.append(m_loss_combine)
    pix_accuracy_train.append(m_acc)
    loss_train_sup.append(np.array(all_loss_train_sup).mean())
    loss_train_unsup.append(np.array(all_loss_train_unsup).mean())

    all_pix_accuracy = []
    all_loss_train_unsup = []
    all_loss_train_sup = []
    all_combine_loss_train = []
    print("loss sup :",loss_sup.item(),"loss unsup :",loss_equiv.item(),"loss :",loss.item(),"iou train :",iou) 
    print("EP:",ep," combine loss train:",m_loss_combine," pixel accuracy between masks ",m_acc)

    ## Evaluate the  model
    model.eval()
    state = ev.eval_model(model,dataloader_val,device=device,num_classes=21)
    iou = state.metrics['mean IoU']
    acc = state.metrics['accuracy']
    loss = state.metrics['CE Loss']
    print('EP:',ep,'iou:',state.metrics['mean IoU'],\
          'Accuracy:',state.metrics['accuracy'],'Loss CE',state.metrics['CE Loss'])
    loss_test.append(loss)
    iou_test.append(iou)
    accuracy_test.append(acc)
    if ep%3==0: # Eval loss equiv and equivariance accuracy for the validation dataset
        m_pix_acc, m_loss_equiv = eval_accuracy_equiv(model,dataloader_val,criterion=criterion_unsupervised,\
                        nclass=21,device=device,Loss=Loss,plot=False,angle_max=30,random_angle=False)
        loss_test_unsup.append(m_loss_equiv)
        pix_accuracy_test.append(m_pix_acc)    
    if ep%5==0: # Eval IoU on rotated input images
        print('VOC Dataset Train')
        d_iou = ev.eval_model_all_angle(model,train=True,batch_size=batch_size,device=device,num_classes=21)
        print('VOC Dataset Val')
        d_iou = ev.eval_model_all_angle(model,train=False,batch_size=batch_size,device=device,num_classes=21)
    ## Save model
    if save_all_ep:
        save_model = model_name+'_ep'+str(ep)+'.pt'
        save = os.path.join(SAVE_DIR,save_model)
        torch.save(model,save)
    else:
        save_model = model_name+'.pt'
        save = os.path.join(SAVE_DIR,save_model)
        torch.save(model,save)

EPOCH 0


/workspace/utils.py:143: RuntimeWarning: invalid value encountered in true_divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)
/workspace/utils.py:145: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))


loss sup : 0.3591289222240448 loss unsup : 0.17465469241142273 loss : 0.3406814932823181 iou train : 0.4209333004030653
EP: 0  combine loss train: 0.8921749784715425  pixel accuracy between masks  0.9156482713200325
EP: 0 iou: 0.4209333004030653 Accuracy: 0.865992344240672 Loss CE 0.4409552488740845
Mean Pixel Accuracy between masks : 0.9229804682717814 Loss Validation : 0.16686472
VOC Dataset Train
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Scores for datasets rotate by 0 degrees:
   mIoU 0.

KeyboardInterrupt: 

In [ ]:
model.eval()
print("EVAL ON TRAIN VOC 2012:")
m_pix_acc, m_loss_equiv = eval_accuracy_equiv(model,dataloader_train_VOC,criterion=criterion_unsupervised,\
                        nclass=21,device=device,Loss=Loss,plot=False,angle_max=30,random_angle=False) 
d_iou = ev.eval_model_all_angle(model,train=True,batch_size=batch_size,device=device,num_classes=21)
print("EVAL ON VAL VOC 2012:")
m_pix_acc, m_loss_equiv = eval_accuracy_equiv(model,dataloader_val,criterion=criterion_unsupervised,\
                        nclass=21,device=device,Loss=Loss,plot=False,angle_max=30,random_angle=False) 
d_iou = ev.eval_model_all_angle(model,train=False,batch_size=batch_size,device=device,num_classes=21)

## Plot

In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("FCN KL semisup. Combine loss train")
plt.plot(combine_loss_train)
plt.xlabel("iterations")
plt.ylabel("Loss")

plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("FCN KL semisup. Equivariance loss train")
plt.plot(loss_train_unsup)
plt.xlabel("iterations")
plt.ylabel("Loss")

plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("FCN KL semisup. CE loss train")
plt.plot(loss_train_sup)
plt.xlabel("iterations")
plt.ylabel("Loss")

plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("FCN KL semisup. Equivariance Accuracy train")
plt.plot(pix_accuracy_train)
plt.xlabel("iterations")
plt.ylabel("Accuracy")

plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("FCN KL semisup. Mean iou train ")
plt.plot(iou_train)
plt.xlabel("iterations")
plt.ylabel("Mean IOU")

plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("FCN KL semisup. Cross entropy loss test")
plt.plot(loss_test)
plt.xlabel("iterations")
plt.ylabel("Loss")

plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("FCN KL semisup. Equivariance loss test")
plt.plot(loss_test_unsup)
plt.xlabel("iterations")
plt.ylabel("Loss")

plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("FCN KL semisup. Equivariance accuracy test")
plt.plot(pix_accuracy_test)
plt.xlabel("iterations")
plt.ylabel("Loss")


plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("FCN KL  semisup. Mean iou test")
plt.plot(iou_test)
plt.xlabel("iterations")
plt.ylabel("Mean IOU")